In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from IPython.utils import io
import torch
from torch import optim
import torch.nn as nn
from torch.nn import functional as F
import random

from transformers import AdamW, AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm

from transformers.models.mt5 import MT5Config, MT5ForConditionalGeneration

sns.set()

In [9]:
# model_repo = 'google/mt5-base'

# config = MT5Config.from_pretrained(model_repo)

d:\ProgramData\anaconda3\venv_test\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
# config.max_length = 40


In [11]:
# tokenizer = AutoTokenizer.from_pretrained(model_repo)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\ProgramData\anaconda3\venv_test\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:
# model.save_pretrained("d:\pyproj\models\mt5\model")
# tokenizer.save_pretrained('d:\\pyproj\\models\\mt5\\tokenizer')

('d:\\pyproj\\models\\mt5\\tokenizer\\tokenizer_config.json',
 'd:\\pyproj\\models\\mt5\\tokenizer\\special_tokens_map.json',
 'd:\\pyproj\\models\\mt5\\tokenizer\\spiece.model',
 'd:\\pyproj\\models\\mt5\\tokenizer\\added_tokens.json',
 'd:\\pyproj\\models\\mt5\\tokenizer\\tokenizer.json')

In [11]:
model_path = "d:\\pyproj\\models\\mt5\\model"
tokenizer_path = 'd:\\pyproj\\models\\mt5\\tokenizer'
max_seq_len = 40
model = MT5ForConditionalGeneration.from_pretrained(model_path)


tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# model = MT5ForConditionalGeneration.from_pretrained(model_repo)
model = model.cuda()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
C:\Users\ezimo\AppData\Roaming\Python\Python311\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
input_test = tokenizer.encode('Токенизируй это',
                             return_tensors='pt',
                             padding='max_length',
                             truncation=True,
                             max_length=max_seq_len)
print(input_test)
tokens = tokenizer.convert_ids_to_tokens(input_test[0])
print(tokens)
print(len(tokens))

tensor([[  3771, 136092,   5436, 100783,   1436,      1,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0]])
['▁То', 'кени', 'зи', 'руй', '▁это', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
40


In [12]:
LANG_TOKEN_MAPPING = {
    'ru': '<ru>',
    'mn': '<mn>'
}

special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}

tokenizer.add_special_tokens(special_tokens_dict)

model.resize_token_embeddings(len(tokenizer))

Embedding(250102, 768)

In [11]:
# special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}

# tokenizer.add_special_tokens(special_tokens_dict)

# model.resize_token_embeddings(len(tokenizer))

Embedding(250102, 768)

In [5]:
np.random.choice(list(LANG_TOKEN_MAPPING.keys()), size=2, replace=False)

array(['ru', 'mn'], dtype='<U2')

In [12]:
def encode_input_str(text, target_lang, tokenizer, seq_len, lang_token_map=LANG_TOKEN_MAPPING):
    
    target_lang_token = lang_token_map[target_lang]
    
    input_ids = tokenizer.encode(
        text = target_lang_token + text,
        return_tensors = 'pt',
        padding = 'max_length',
        truncation = True,
        max_length = seq_len
        )
    
    return input_ids[0]

def encode_target_str(text, tokenizer, seq_len):
    
    input_ids = tokenizer.encode(
        text = text,
        return_tensors = 'pt',
        padding = 'max_length',
        truncation = True,
        max_length = seq_len
        )
    
    return input_ids[0]

def format_translation_data(translations, lang_token_map, tokenizer, seq_len=128):
    
    langs = list(lang_token_map.keys())
    input_lang, target_lang = np.random.choice(langs, size=2, replace=False)
    
    input_text = translations[input_lang]
    target_text = translations[target_lang]
    
#     print(input_lang, input_text)
#     print(target_lang, target_text)
    
    if input_text is None or target_text is None:
        return None
    
    input_token_ids = encode_input_str(input_text, target_lang, tokenizer, seq_len, lang_token_map)
    target_token_ids = encode_target_str(target_text, tokenizer, seq_len)
    
    return input_token_ids, target_token_ids

def transform_batch(batch, lang_token_map, tokenizer):
    inputs = []
    targets = []
    
    for translation_set in batch:
        formatted_data = format_translation_data(translation_set, lang_token_map, tokenizer, max_seq_len)
        
#         print(formatted_data)
        
        if formatted_data is None:
            continue
        
        input_ids, target_ids = formatted_data
        inputs.append(input_ids.unsqueeze(0))
        targets.append(target_ids.unsqueeze(0))
    
    batch_input_ids = torch.cat(inputs).cuda()
    batch_target_ids = torch.cat(targets).cuda()
    
    return batch_input_ids, batch_target_ids

def get_data(dataset, lang_token_map, tokenizer, batch_size=32):
    np.random.shuffle(dataset)
    for i in range(0, len(dataset), batch_size):
        raw_batch = dataset[i:i+batch_size]
        
        yield(transform_batch(raw_batch, lang_token_map, tokenizer))

In [14]:
import pandas as pd
import random
# from translate.storage.tmx import tmxfile
from easygoogletranslate import EasyGoogleTranslate

In [15]:
df_total = pd.read_csv(r'd:\pyproj\models\mt5\mn_ru.csv')
df_total.head()

,ru,mn
0,Один раз в жизни я делаю хорошее дело... И оно...,Амьдралдаа нэг удаа буян хийдэг... Тэгээд дэми...
1,Давайте что-нибудь попробуем!,Ямар нэг зүйл туршиж үзье!
2,Давайте что-нибудь попробуем.,Ямар нэг юм оролдоод үзье.
3,Мне пора идти спать.,Миний унтах цаг боллоо.
4,Мне нужно идти спать.,Би орондоо орох ёстой.


In [16]:
df_total.shape

(82920, 2)

In [17]:
dataset = []

for i in range(len(df_total)):
    dataset.append({'mn': df_total.loc[i]['mn'], 'ru': df_total.loc[i]['ru']})

train_dataset = dataset[:82000]
test_dataset = dataset[82000:]

In [18]:
dataset[:5]

[{'mn': 'Амьдралдаа нэг удаа буян хийдэг... Тэгээд дэмий юм.',
  'ru': 'Один раз в жизни я делаю хорошее дело... И оно бесполезно. '},
 {'mn': 'Ямар нэг зүйл туршиж үзье!', 'ru': 'Давайте что-нибудь попробуем! '},
 {'mn': 'Ямар нэг юм оролдоод үзье.', 'ru': 'Давайте что-нибудь попробуем. '},
 {'mn': 'Миний унтах цаг боллоо.', 'ru': 'Мне пора идти спать. '},
 {'mn': 'Би орондоо орох ёстой.', 'ru': 'Мне нужно идти спать. '}]

In [19]:
len(dataset)

82920

In [20]:
k = random.randint(0, 50020)
print(k)
dataset[k]

35296


{'mn': 'Бид дээш гарах тусам хүйтрэх болно.',
 'ru': 'Чем выше мы поднимаемся, тем холоднее становится. '}

In [ ]:
# Train

In [21]:
n_epochs = 3
batch_size = 20
print_freq = 100

lr = 5e-4

checkpoint_freq = 1000

n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

print("n_batches", n_batches)
print("total_steps", total_steps)
print("n_warmup_steps", n_warmup_steps)

n_batches 4100
total_steps 12300
n_warmup_steps 123


In [22]:
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, n_warmup_steps, total_steps)

losses = []
test_losses = []

C:\Users\ezimo\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
def eval_model(model, dataset, max_iters=8):
    test_generator = get_data(dataset, LANG_TOKEN_MAPPING, tokenizer, batch_size)
    eval_losses = []
    with torch.no_grad():
        for i, (input_batch, label_batch) in enumerate(test_generator):
            if i >= max_iters:
                break
            
            model_out = model.forward(input_ids = input_batch, labels = label_batch)
            eval_losses.append(model_out.loss.item())

    return np.mean(eval_losses)

In [24]:
best_test_loss = float('inf')

In [25]:
model_path = r'd:\pyproj\models\mt5\model.safetensors'
for epoch_idx in range(n_epochs):
    data_generator = get_data(train_dataset, LANG_TOKEN_MAPPING, tokenizer, batch_size)

    for batch_idx, (input_batch, label_batch) in tqdm(enumerate(data_generator), total=n_batches):
        optimizer.zero_grad()
        

        model_out = model.forward(
            input_ids = input_batch,
            labels = label_batch)

        loss = model_out.loss
        losses.append(loss.item())
    
        loss.backward()

        optimizer.step()
        scheduler.step()

        # Print training update info
        if (batch_idx + 1) % print_freq == 0:
            avg_loss = np.mean(losses[-print_freq:])
            print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {:.6f}'.format(
              epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))

        if (batch_idx + 1) % checkpoint_freq == 0:
            test_loss = eval_model(model, test_dataset)
            test_losses.append(test_loss)
            print('Test loss {:.3f}'.format(test_loss))
            if best_test_loss > test_loss:
                print('Saving model with test loss of {:.3f}'.format(test_loss))
                torch.save(model.state_dict(), model_path)
                best_test_loss = test_loss

    torch.save(model.state_dict(), model_path)

  2%|█▉                                                                           | 100/4100 [02:24<1:37:06,  1.46s/it]

Epoch: 1 | Step: 100 | Avg. loss: 14.084 | lr: 0.000407


  5%|███▊                                                                         | 200/4100 [04:49<1:33:36,  1.44s/it]

Epoch: 1 | Step: 200 | Avg. loss: 1.774 | lr: 0.000497


  7%|█████▋                                                                       | 300/4100 [07:15<1:33:17,  1.47s/it]

...

Epoch: 3 | Step: 4000 | Avg. loss: 0.229 | lr: 0.000004


 98%|██████████████████████████████████████████████████████████████████████████▏ | 4000/4100 [1:40:33<04:07,  2.48s/it]

Test loss 0.252


100%|████████████████████████████████████████████████████████████████████████████| 4100/4100 [1:43:03<00:00,  1.51s/it]

Epoch: 3 | Step: 4100 | Avg. loss: 0.225 | lr: 0.000000


In [2]:
model_path = "d:\\pyproj\\models\\mt5\\model"
tokenizer_path = 'd:\\pyproj\\models\\mt5\\tokenizer'
max_seq_len = 128

model = MT5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

LANG_TOKEN_MAPPING = {
    'ru': '<ru>',
    'mn': '<mn>'
}

special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}

tokenizer.add_special_tokens(special_tokens_dict)

model.resize_token_embeddings(len(tokenizer))



model.load_state_dict(torch.load(r'd:\pyproj\models\mt5\model_.safetensors'))
model = model.cuda()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
C:\Users\ezimo\AppData\Roaming\Python\Python311\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:

df_total = pd.read_csv(r'd:\pyproj\models\mt5\mn_ru.csv')
df_total.head()
dataset = []

for i in range(len(df_total)):
    dataset.append({'mn': df_total.loc[i]['mn'], 'ru': df_total.loc[i]['ru']})
len(dataset)
# train_dataset = dataset[:32600]
# test_dataset = dataset[32600:32900]

82920

In [6]:
train_dataset = dataset[:82000]
test_dataset = dataset[82000:]

In [7]:
len(test_dataset)

920

In [10]:
k = random.randint(0, 920)
test_dataset[k]

{'mn': 'Удахгүй бид чамайг шоронд хийж чадна.',
 'ru': 'Скоро мы сможем посадить вас в тюрьму. '}

In [13]:
max_seq_len = 128
k = random.randint(0, 520)
test_sentence = test_dataset[k]['mn']
print(k)
# print('Raw input text:', test_sentence)

input_ids = encode_input_str(
    text = test_sentence,
    target_lang = 'ru',
    tokenizer = tokenizer,
    seq_len = max_seq_len,
    lang_token_map = LANG_TOKEN_MAPPING)

input_ids = input_ids.unsqueeze(0).cuda()

print(test_dataset[k])
print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

274
{'mn': 'Би түлхүүрээ алдсан.', 'ru': 'Я свой ключ потерял. '}
Truncated input text: <ru> Би түлхүүрээ алдсан.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [14]:
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=3)

# print(output_tokens)

for token_set in output_tokens:
    print(tokenizer.decode(token_set, skip_special_tokens=True))

C:\Users\ezimo\AppData\Roaming\Python\Python311\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Яблоко, которое я потерял.
Яблоко, которое мы потеряли.
Яблоко, которое я потерял, сломано.


In [15]:
def translate(text, lang_from, lang_to):
    # test_sentence = test_dataset[16][lang_from]
    # print('Raw input text:', test_sentence)

    input_ids = encode_input_str(
      text = text,
      target_lang = lang_to,
      tokenizer = tokenizer,
        seq_len=128,
      # seq_len = model.config.max_length,
      lang_token_map = LANG_TOKEN_MAPPING)

    input_ids = input_ids.unsqueeze(0).cuda()
    # res = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0]))

    output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=1)

    res = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    return res

def translate_to_mogolian(text):
    return translate(text, 'ru', 'mn')

def translate_to_russian(text):
    return translate(text, 'mn', 'ru')

In [27]:
k = random.randint(0, 920)
print(test_dataset[k])
translate_to_russian(test_dataset[k]['mn'])

{'mn': 'Тэд нохой, муур шиг тулалддаг.', 'ru': 'Они цапаются как кошка с собакой. '}


'Они сталкиваются с собакой как с котом.'

In [17]:
translate_to_russian('Усанд сэлэх нь таны эрүүл мэндэд тустай. Хүний бие хэдэн тэрбум жижиг эсээс бүрддэг.')

'Лёд полезен для здоровья. Человек состоит из нескольких м'